# Conexcióna Bigquery

In [1]:

import os
import sys
import pathlib

sys.path.append(
    os.path.join(pathlib.Path(os.getcwd()).parent)
)
import pandas as pd
from datetime import datetime
import plotly.graph_objects as go
import plotly.express as px
from utils.bq_conn import bq_conn_file

bq_client = bq_conn_file(project_id="dsa-miad-365616")

# Configurar opciones de visualización para mostrar todas las columnas
pd.set_option('display.max_columns', None)

In [2]:
query = '''
SELECT
    *
FROM `Proyecto.DatosSecopI`
ORDER BY anno_cargue_secop DESC
'''
secop_I = bq_client.execute_query(query=query)
secop_I

,uid,anno_cargue_secop,id_modalidad,modalidad_de_contratacion,estado_del_proceso,id_objeto_a_contratar,detalle_del_objeto_a_contratar,tipo_de_contrato,municipio_de_obtencion,municipios_ejecucion_,departamento_ejecucion,numero_de_proceso,numero_de_contrato,cuantia_proceso,id_grupo,nombre_grupo,id_familia,nombre_familia,id_clase,nombre_clase,id_adjudicacion,tipo_identifi_del_contratista,identificacion_del_contratista,nom_razon_social_contratista,tipo_doc_representante_legal,identific_representante_legal,plazo_de_ejec_del_contrato_,dias_ejec_final_contrato,rango_de_ejec_del_contrato,prorrogado,tiempo_adiciones_en_dias,cuantia_contrato,con_adicion_pptal,valor_total_de_adiciones,valor_contrato_con_adiciones,moneda,objeto_del_contrato_a_la,ruta_proceso_en_secop_i
0,191389136418143424,2019,13,contratacion minima cuantia,celebrado,50000000,contratar servicio suministro elementos materi...,suministro,facatativa,facatativa,cundinamarca,CPMMSFFAMCNo01DE2019,CPMMSFFAMCNO01DE2019,50000000,E,productos de uso final,5022,productos de cereales legumbres,502213,harina productos de molinos,8143424,nit de persona juridica,900791326,inversiones ocazo,cedula de ciudadania,1022363447,330,330,D,0,0,50000000,0,0,50000000,peso colombiano,contratar compra elementos materia prima para ...,{'url': 'https://www.contratos.gov.co/consulta...
1,1912101041779226352,2019,12,contratacion directa ley de,celebrado,53000000,adquisicion uniformes para dotar personal cuer...,prestacion de servicios,bogota,bogota,bogota,DIRGENERALCONTRATACIONDIRECTA184DE2019,184DE2019,22980096902,E,productos de uso final,5310,ropa,531027,uniformes,9226352,nit de persona juridica,860020227,fondo rotatorio de la policia forpo,nit de persona juridica,860020227,660,690,D,1,30,22980096902,0,0,22980096902,peso colombiano,adquisicion uniformes para dotar personal cuer...,{'url': 'https://www.contratos.gov.co/consulta...
2,1912101443259265538,2019,12,contratacion directa ley de,celebrado,55000000,contratar diseno produccion tarjetas presentac...,prestacion de servicios,bogota,bogota,bogota,DIRGENERALCONTRATACIONDIRECTA188DE2019,188DE2019,35998452,E,productos de uso final,5510,medios impresos,551015,publicaciones impresas,9265538,nit de persona juridica,830001113,imprenta nacional de colombia,nit de persona juridica,830001113,31,71,D,1,40,35998452,0,0,35998452,peso colombiano,contratar diseno produccion tarjetas presentac...,{'url': 'https://www.contratos.gov.co/consulta...
3,191294985788644918,2019,12,contratacion directa ley de,liquidado,72000000,prestar servicio mantenimiento preventivo corr...,prestacion de servicios,bogota,bogota,bogota,DIRGENERALCONTRATACIONDIRECTA106DE2019,106DE2019,16767536,F,servicios,7210,servicios de mantenimiento reparaciones de con...,721015,servicios de apoyo para la construccion,8644918,nit de persona juridica,860025639,mitsubishi electric de colombia limitada,nit de persona juridica,860025639,170,170,D,0,0,16767536,0,0,16767536,peso colombiano,prestar servicio mantenimiento preventivo corr...,{'url': 'https://www.contratos.gov.co/consulta...
4,191988590538059861,2019,19,asociacion publico privada,celebrado,73000000,desintegracion vehiculos automotores estado in...,no encontrado en hoja proceso,bogota,bogota,bogota,CONVENIODEASOCIACIÓN231DE2018,231DE2018,0,F,servicios,7318,servicios de maquinado procesado,731810,servicios de maquinado,8059861,nit de persona juridica,806011019,recuperaciones naranjo recycling sas,cedula de ciudadania,57439563,360,360,D,0,0,0,0,0,0,peso colombiano,desintegracion vehiculos automotores estado in...,{'url': 'https://www.contratos.gov.co/consulta...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26548,151240452323758751,2015,12,contratacion directa ley de,celebrado,94000000,motivo celebracion guardian requiere contratar...,prestacion de servicios,tierralta,tierralta,cordoba,32403015,ordendeservicio,1220210,F,servicios,9412,club

## Tablas de identificación

1. Modalidad de contratación

In [3]:
tabla_modalidad = secop_I.groupby(['id_modalidad', 'modalidad_de_contratacion']).size().reset_index(name='cantidad')
tabla_modalidad

,id_modalidad,modalidad_de_contratacion,cantidad
0,1,licitacion publica,8
1,9,subasta,246
2,11,seleccion abreviada de menor cuantia ley de,103
3,12,contratacion directa ley de,2700
4,13,contratacion minima cuantia,23491
5,18,seleccion abreviada del literal del numeral de...,4
6,19,asociacion publico privada,1


In [4]:
fig = px.pie(tabla_modalidad, values='cantidad', names='modalidad_de_contratacion', title='Modalidad de contratación')
fig.show()

2. Objeto a contratar 

In [5]:
tabla_objeto_a_contratar = secop_I.groupby(['id_objeto_a_contratar', 'detalle_del_objeto_a_contratar']).size().reset_index(name='cantidad')
tabla_objeto_a_contratar

,id_objeto_a_contratar,detalle_del_objeto_a_contratar,cantidad
0,10000000,adqucision material veterinario para epmsc mocoa,1
1,10000000,adquirir alimento concentrado material arneses...,1
2,10000000,adquirir alimento concentrado material veterin...,1
3,10000000,adquirir alimento concentrado material veterin...,1
4,10000000,adquirir alimento concentrado melaza para tern...,1
...,...,...,...
20721,95000000,establecimiento penitenciario mediana segurida...,1
20722,95000000,mantenimiento porcicultura,1
20723,95000000,para arrendamiento espacios fisicos establecim...,1
20724,95000000,proceso seleccion para contratar arrendamiento...,1


3. Grupo: Categorización inicial del bien o servicio definido en el proceso de compra, de acuerdo con sus características principales.

In [6]:
tabla_grupo = secop_I.groupby(['id_grupo', 'nombre_grupo']).size().reset_index(name='cantidad')
tabla_grupo

,id_grupo,nombre_grupo,cantidad
0,A,material vivo animal vegetal,1518
1,B,materias primas,2214
2,C,maquinaria herramientas equipo industrial vehi...,943
3,D,componentes suministros,1241
4,E,productos de uso final,15875
5,F,servicios,4729
6,G,terrenos edificios estructuras vias,33


In [17]:
fig = px.pie(tabla_grupo, values='cantidad', names='nombre_grupo', title='Categorización Grupo')
fig.show()

4. Familia

In [10]:
tabla_familia = secop_I.groupby(['id_familia', 'nombre_familia']).size().reset_index(name='cantidad')
tabla_familia

,id_familia,nombre_familia,cantidad
0,1010,animales vivos,91
1,1011,productos para animales domesticos,299
2,1012,comida de animales,766
3,1015,semillas bulbos plantulas esquejes,27
4,1016,productos de floricultura silvicultura,4
...,...,...,...
295,9510,parcelas de tierra,3
296,9512,estructuras edificios permanentes,17
297,9513,estructuras edificios moviles,2
298,9514,estructuras edificios prefabricados,1


In [30]:
fig =px.treemap(tabla_familia, path=[px.Constant('familia'),'nombre_familia'], values='cantidad',
                  color='id_familia', title="Categorización Familia")
fig.show()

5. Clase

In [13]:
tabla_clase = secop_I.groupby(['id_clase', 'nombre_clase']).size().reset_index(name='cantidad')
tabla_clase

,id_clase,nombre_clase,cantidad
0,101015,animales de granja,61
1,101016,pajaros aves de corral,22
2,101017,peces,8
3,101113,tratamientos para los animales domesticos acce...,299
4,101215,pienso para ganado,88
...,...,...,...
954,951217,edificios estructuras publicos,13
955,951221,edificios estructuras de acomodaciones,1
956,951316,edificios estructuras comerciales industriales...,2
957,951415,edificios estructuras prefabricados para fincas,1


In [31]:
fig =px.treemap(tabla_clase, path=[px.Constant('clase'),'nombre_clase'], values='cantidad',
                  color='id_clase', title="Categorización Calse")
fig.show()

### Departamento

In [123]:
depart = secop_I.groupby(['departamento_ejecucion']).size().reset_index(name='cantidad')
fig = px.pie(depart, values='cantidad', names='departamento_ejecucion', title='Contratos por Departamento')
fig.show()


In [122]:
data_II = secop_I.groupby(['departamento_ejecucion','estado_del_proceso']).size().reset_index(name='cantidad')
fig = px.treemap(data_II, path=['departamento_ejecucion','estado_del_proceso'], values='cantidad',
                  color='cantidad', title="Estado de Contratos por Departamento")
fig.show()

### Distribución Acumulativa Empírica (Empirical Cumulative Distribution Plot, ECDF) 
es una herramienta para visualizar y analizar la distribución de una variable. En lugar de mostrar la densidad de probabilidad como en un histograma, el ECDF muestra la proporción acumulativa de observaciones en o por debajo de un valor específico. El eje x representa los valores de la variable y el eje y muestra la proporción acumulativa de observaciones, que varía entre 0 y 1.



In [173]:
# cuantia = secop_I.groupby(['estado_del_proceso','cuantia_proceso']).size().reset_index(name='cantidad')

fig = px.ecdf(secop_I, x="valor_contrato_con_adiciones",color="nombre_grupo", 
              range_x=[0,2000000000],markers=True, title="Distribución del valor del contrato dependiendo su Grupo")
fig.show()

### Tiempo de ejecución

In [93]:
data_I = secop_I.groupby(['id_grupo', 'nombre_grupo','plazo_de_ejec_del_contrato_','departamento_ejecucion', 'municipios_ejecucion_']).size().reset_index(name='cantidad')
data_I.head()

,id_grupo,nombre_grupo,plazo_de_ejec_del_contrato_,departamento_ejecucion,municipios_ejecucion_,cantidad
0,A,material vivo animal vegetal,0,antioquia,apartado,3
1,A,material vivo animal vegetal,0,antioquia,bello,1
2,A,material vivo animal vegetal,0,antioquia,caucasia,1
3,A,material vivo animal vegetal,0,antioquia,puerto berrio,3
4,A,material vivo animal vegetal,0,antioquia,santa rosa de osos,2


In [161]:
fig = px.sunburst(data_I, path=['nombre_grupo'], values='cantidad',
                  color='plazo_de_ejec_del_contrato_', range_color=[0,200])
fig.show()

In [107]:
fig = px.treemap(data_I, path=['departamento_ejecucion', 'municipios_ejecucion_'], values='cantidad',
                  color='plazo_de_ejec_del_contrato_', range_color=[0,400])
fig.show()

### Prorrogados

In [187]:
data_ad = secop_I.groupby(['tipo_de_contrato','prorrogado']).size().reset_index(name='cantidad')
fig = px.pie(data_ad[data_ad.prorrogado==1], values='cantidad', names='tipo_de_contrato', title='% de Tipo de contratos Prorrogados')
fig.show()

In [188]:
data_ad = secop_I.groupby(['nombre_grupo','prorrogado']).size().reset_index(name='cantidad')
fig = px.pie(data_ad[data_ad.prorrogado==1], values='cantidad', names='nombre_grupo', title='% de Grupo Prorrogados')
fig.show()

In [196]:
data_ad = secop_I.groupby(['nombre_familia','prorrogado']).size().reset_index(name='cantidad')
fig = px.pie(data_ad[(data_ad.prorrogado==1) & (data_ad.nombre_familia != "no definido") & (data_ad.cantidad > 10)], values='cantidad', names='nombre_familia', title='% de Familia Prorrogados')
fig.show()

In [199]:
data_ad = secop_I.groupby(['nombre_clase','prorrogado']).size().reset_index(name='cantidad')
fig = px.pie(data_ad[(data_ad.prorrogado==1) & (data_ad.nombre_clase != "no definido")& (data_ad.cantidad > 10)], values='cantidad', names='nombre_clase', title='% de Clase Prorrogados')
fig.show()

In [207]:
data_ad = secop_I.groupby(['modalidad_de_contratacion','estado_del_proceso','tipo_de_contrato','prorrogado']).size().reset_index(name='cantidad')
fig = px.treemap(data_ad[data_ad.prorrogado==1], 
                 path=['estado_del_proceso', 'modalidad_de_contratacion', 'tipo_de_contrato'], 
                 values='cantidad',
                 color='cantidad',
                 title="Mapa de Contratos Prorrogados dependiendo su Estado, Modalidad y Tipo de Contrato")
fig.show()

### Adición Presupuestal

In [208]:
data_ad = secop_I.groupby(['tipo_de_contrato','con_adicion_pptal']).size().reset_index(name='cantidad')
fig = px.pie(data_ad[data_ad.con_adicion_pptal==1], values='cantidad', names='tipo_de_contrato', title='% de Tipo de contratos con Adición Presupuestal')
fig.show()

In [209]:
data_ad = secop_I.groupby(['nombre_grupo','con_adicion_pptal']).size().reset_index(name='cantidad')
fig = px.pie(data_ad[data_ad.con_adicion_pptal==1], values='cantidad', names='nombre_grupo', title='% de Grupo con Adición Presupuestal')
fig.show()

In [211]:
data_ad = secop_I.groupby(['nombre_familia','con_adicion_pptal']).size().reset_index(name='cantidad')
fig = px.pie(data_ad[(data_ad.con_adicion_pptal==1) & (data_ad.nombre_familia != "no definido") & (data_ad.cantidad > 15)], values='cantidad', names='nombre_familia', title='% de Familia con Adición Presupuestal')
fig.show()

In [213]:
data_ad = secop_I.groupby(['nombre_clase','con_adicion_pptal']).size().reset_index(name='cantidad')
fig = px.pie(data_ad[(data_ad.con_adicion_pptal==1) & (data_ad.nombre_clase != "no definido")& (data_ad.cantidad > 15)], values='cantidad', names='nombre_clase', title='% de Clase con Adición Presupuestal')
fig.show()

In [215]:
data_ad = secop_I.groupby(['modalidad_de_contratacion','estado_del_proceso','tipo_de_contrato','con_adicion_pptal']).size().reset_index(name='cantidad')
fig = px.treemap(data_ad[data_ad.con_adicion_pptal==1], 
                 path=['estado_del_proceso', 'modalidad_de_contratacion', 'tipo_de_contrato'], 
                 values='cantidad',
                 color='cantidad',
                 title="Mapa de Contratos con Adición Presupuestal dependiendo su Estado, Modalidad y Tipo de Contrato")
fig.show()

In [171]:
secop_I.head()

,uid,anno_cargue_secop,id_modalidad,modalidad_de_contratacion,estado_del_proceso,id_objeto_a_contratar,detalle_del_objeto_a_contratar,tipo_de_contrato,municipio_de_obtencion,municipios_ejecucion_,departamento_ejecucion,numero_de_proceso,numero_de_contrato,cuantia_proceso,id_grupo,nombre_grupo,id_familia,nombre_familia,id_clase,nombre_clase,id_adjudicacion,tipo_identifi_del_contratista,identificacion_del_contratista,nom_razon_social_contratista,tipo_doc_representante_legal,identific_representante_legal,plazo_de_ejec_del_contrato_,dias_ejec_final_contrato,rango_de_ejec_del_contrato,prorrogado,tiempo_adiciones_en_dias,cuantia_contrato,con_adicion_pptal,valor_total_de_adiciones,valor_contrato_con_adiciones,moneda,objeto_del_contrato_a_la,ruta_proceso_en_secop_i
0,191389136418143424,2019,13,contratacion minima cuantia,celebrado,50000000,contratar servicio suministro elementos materi...,suministro,facatativa,facatativa,cundinamarca,CPMMSFFAMCNo01DE2019,CPMMSFFAMCNO01DE2019,50000000,E,productos de uso final,5022,productos de cereales legumbres,502213,harina productos de molinos,8143424,nit de persona juridica,900791326,inversiones ocazo,cedula de ciudadania,1022363447,330,330,D,0,0,50000000,0,0,50000000,peso colombiano,contratar compra elementos materia prima para ...,{'url': 'https://www.contratos.gov.co/consulta...
1,1912101041779226352,2019,12,contratacion directa ley de,celebrado,53000000,adquisicion uniformes para dotar personal cuer...,prestacion de servicios,bogota,bogota,bogota,DIRGENERALCONTRATACIONDIRECTA184DE2019,184DE2019,22980096902,E,productos de uso final,5310,ropa,531027,uniformes,9226352,nit de persona juridica,860020227,fondo rotatorio de la policia forpo,nit de persona juridica,860020227,660,690,D,1,30,22980096902,0,0,22980096902,peso colombiano,adquisicion uniformes para dotar personal cuer...,{'url': 'https://www.contratos.gov.co/consulta...
2,1912101443259265538,2019,12,contratacion directa ley de,celebrado,55000000,contratar diseno produccion tarjetas presentac...,prestacion de servicios,bogota,bogota,bogota,DIRGENERALCONTRATACIONDIRECTA188DE2019,188DE2019,35998452,E,productos de uso final,5510,medios impresos,551015,publicaciones impresas,9265538,nit de persona juridica,830001113,imprenta nacional de colombia,nit de persona juridica,830001113,31,71,D,1,40,35998452,0,0,35998452,peso colombiano,contratar diseno produccion tarjetas presentac...,{'url': 'https://www.contratos.gov.co/consulta...
3,191294985788644918,2019,12,contratacion directa ley de,liquidado,72000000,prestar servicio mantenimiento preventivo corr...,prestacion de servicios,bogota,bogota,bogota,DIRGENERALCONTRATACIONDIRECTA106DE2019,106DE2019,16767536,F,servicios,7210,servicios de mantenimiento reparaciones de con...,721015,servicios de apoyo para la construccion,8644918,nit de persona juridica,860025639,mitsubishi electric de colombia limitada,nit de persona juridica,860025639,170,170,D,0,0,16767536,0,0,16767536,peso colombiano,prestar servicio mantenimiento preventivo corr...,{'url': 'https://www.contratos.gov.co/consulta...
4,191988590538059861,2019,19,asociacion publico privada,celebrado,73000000,desintegracion vehiculos automotores estado in...,no encontrado en hoja proceso,bogota,bogota,bogota,CONVENIODEASOCIACIÓN231DE2018,231DE2018,0,F,servicios,7318,servicios de maquinado procesado,731810,servicios de maquinado,8059861,nit de persona juridica,806011019,recuperaciones naranjo recycling sas,cedula de ciudadania,57439563,360,360,D,0,0,0,0,0,0,peso colombiano,desintegracion vehiculos automotores estado in...,{'url': 'https://www.contratos.gov.co/consulta...


In [157]:
# query = '''
# SELECT
#     *
# FROM `Proyecto.DatosSecopII`
# ORDER BY estado_contrato DESC
# '''
# secop_II = bq_client.execute_query(query=query)
# secop_II.head(3)